In [ ]:
Conversation opened. 1 unread message.

Skip to content
Using Gmail with screen readers
1 of 20,559
Engine
Inbox

JOSE THOMAS
2:53 PM (0 minutes ago)
to me

import pandas as pd
import os, shutil, glob
import glob
from datetime import datetime

file_list1 = glob.glob('D:/CM_NSE/UDIFF/*.csv')

final_df1 = pd.DataFrame() #empty dataframe

for csv_file in file_list1:
    df = pd.read_csv(csv_file)
    df.columns = df.columns.str.replace(' ', '')
    df = df.rename(columns={'TradDt': 'TIMESTAMP'})

    df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'])
    #df.set_index(['TIMESTAMP'], inplace=True)

    df_trim = df.apply(lambda x: x.str.strip() if x.dtype == 'object' else x)
    new_df = df_trim[df_trim['SctySrs'].isin(['EQ'])] 
    final_df1 = pd.concat([final_df1, new_df])

final_df1=final_df1[['TIMESTAMP','TckrSymb','OpnPric','HghPric','LwPric','ClsPric']]
final_df1.set_index(['TIMESTAMP'], inplace=True)
final_df1.sort_index(inplace=True)
final_df1 = final_df1.rename(columns={'TradDt': 'TIMESTAMP','TckrSymb':'SYMBOL','OpnPric':'OPEN','HghPric':'HIGH','LwPric':'LOW','ClsPric':'CLOSE'})


dfs1 = {}

for name, group in final_df1.groupby('SYMBOL'):
    df_ts = group[['SYMBOL','OPEN','HIGH','LOW','CLOSE']]
    # add the dataframe to the dictionary
    dfs1[name] = df_ts
    
    
import pandas as pd

# Load the CSV file
csv_file_path = 'D:/CM_NSE/FNO.csv'  
csv_data = pd.read_csv(csv_file_path)
symbols_from_csv = set(csv_data['SYMBOL'])



# Filter the dictionary based on symbols in the CSV
dfs1_filtered = {symbol: df for symbol, df in dfs1.items() if symbol in symbols_from_csv}

for symbol, data in dfs1_filtered.items():
    # Shift the 'OPEN' column by -1 to get the next day's opening price
    CLOSE = data['CLOSE']
    
    # Use the latest available closing value as the 'last day's close'
    last_day_close = data['CLOSE'].iloc[-1]  # This fetches the last available closing value
    
    # Calculate the percentage difference between next day's open and the last available close
    data['Percentage_Difference'] = round((( last_day_close - CLOSE ) / CLOSE) * 100,2)
    # Create a column for the latest closing price
    data['Latest_Close_Price'] = last_day_close
    
    
    
    # Update the dictionary with the modified DataFrame
    dfs1_filtered[symbol] = data





std_dev_multiplier = 2  # Multiplier for the standard deviation

for symbol, data in dfs1_filtered.items():
    df = pd.DataFrame()
    df = pd.DataFrame(data)
    
    # Calculate 5-period simple moving average (SMA) of the bar's midpoints
    df['midpoint'] = (df['HIGH'] + df['LOW']) / 2
    df['SMA_5'] = df['midpoint'].rolling(window=5).mean()

    # Calculate 34-period simple moving average (SMA) of the bar's midpoints
    df['SMA_34'] = df['midpoint'].rolling(window=34).mean()

    # Calculate the Awesome Oscillator (AO) as the difference between the 5-period SMA and 34-period SMA
    df['Awesome_Oscillator'] = df['SMA_5'] - df['SMA_34']

    # Optional: You can also calculate a moving average of the Awesome Oscillator for smoother signals
    # Here, we'll use a 5-period simple moving average
    df['AO_SMA'] = df['Awesome_Oscillator'].rolling(window=5).mean()

    # Generate buy/sell signals based on Awesome Oscillator
    df['Buy_Signal'] = (df['Awesome_Oscillator'] > df['AO_SMA']) & (df['Awesome_Oscillator'].shift(1) < df['AO_SMA'].shift(1))
    df['Sell_Signal'] = (df['Awesome_Oscillator'] < df['AO_SMA']) & (df['Awesome_Oscillator'].shift(1) > df['AO_SMA'].shift(1))

    # Convert boolean signals to integers (0 or 1)
    df['AO_BUY'] = df['Buy_Signal'].astype(int)
    df['AO_SELL'] = df['Sell_Signal'].astype(int)
    
   
    # Drop intermediate columns if not needed
    df.drop(['midpoint', 'SMA_5', 'SMA_34', 'AO_SMA'], axis=1, inplace=True)

    
    
    
    
    
    
    df['5DMA'] = df['CLOSE'].rolling(window=5).mean()
    df['8DMA'] = df['CLOSE'].rolling(window=8).mean()
    df['13DMA'] = df['CLOSE'].rolling(window=13).mean()
    df['21DMA'] = df['CLOSE'].rolling(window=21).mean()
    df['20DMA'] = df['CLOSE'].rolling(window=20).mean()
    df['50DMA'] = df['CLOSE'].rolling(window=50).mean()
     
        
    df['Bullish_Engulfing'] = ((df['CLOSE'].shift(1) < df['OPEN'].shift(1))  & (df['CLOSE'] > df['OPEN'].shift(1)) & (df['OPEN'] < df['CLOSE'].shift(1)) &  (df['CLOSE'] < df['20DMA'].shift(1))).astype(int) 
    df['Bearish_Engulfing'] = ((df['CLOSE'].shift(1) > df['OPEN'].shift(1))  & (df['CLOSE'] < df['OPEN'].shift(1)) & (df['OPEN'] > df['CLOSE'].shift(1)) &  (df['CLOSE'] > df['20DMA'].shift(1))).astype(int) 



    # Calculate the rolling mean (middle line)
    df['MA_BB'] = df['CLOSE'].rolling(window=20).mean()
    
    
    df['52week_high'] =df['CLOSE'].shift(1).rolling(window=240).max()
    df['52week_low'] = df['CLOSE'].shift(1).rolling(window=240).min()
    
    df['30day_high'] = df['CLOSE'].shift(1).rolling(window=30).max()
    df['30day_low'] = df['CLOSE'].shift(1).rolling(window=30).min()

    
    df['7day_high'] = df['CLOSE'].shift(1).rolling(window=7).max()
    df['7day_low'] =df['CLOSE'].shift(1).rolling(window=7).min()
    
    df['10day_high'] = df['CLOSE'].shift(1).rolling(window=10).max()
    df['10day_low'] =df['CLOSE'].shift(1).rolling(window=10).min()
        

    
    
    
    # Calculate the rolling standard deviation
    df['STD_BB'] = df['CLOSE'].rolling(window=20).std()


    # Calculate the upper and lower Bollinger Bands
    df['Upper_Band_BB'] = df['MA_BB'] + std_dev_multiplier * df['STD_BB']
    df['Lower_Band_BB'] = df['MA_BB'] - std_dev_multiplier * df['STD_BB']   
    
    def wilder_smoothing(data, window):
        alpha = 1 / window
        smoothed = [data[0]]

        for i in range(1, len(data)):
            smoothed_value = alpha * data[i] + (1 - alpha) * smoothed[-1]
            smoothed.append(smoothed_value)

        return pd.Series(smoothed, index=data.index)

        # Assuming 'gain' and 'loss' are pandas Series representing the gain and loss values
    window = 7
    df['delta'] = df['CLOSE'].diff(1)

    df['gain'] = (df['delta'].where(df['delta'] > 0, 0)).fillna(0)
    df['loss'] = (-df['delta'].where(df['delta'] < 0, 0)).fillna(0)

    avg_gain = wilder_smoothing(df['gain'], window)
    avg_loss = wilder_smoothing(df['loss'], window)

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    df['RSI_7'] = rsi
    df['RSI_7_ma'] = df['RSI_7'].rolling(window=7).mean()

    
    def wilder_smoothing(data, window):
        alpha = 1 / window
        smoothed = [data[0]]

        for i in range(1, len(data)):
            smoothed_value = alpha * data[i] + (1 - alpha) * smoothed[-1]
            smoothed.append(smoothed_value)

        return pd.Series(smoothed, index=data.index)

    window = 14
    df['delta'] = df['CLOSE'].diff(1)

    df['gain'] = (df['delta'].where(df['delta'] > 0, 0)).fillna(0)
    df['loss'] = (-df['delta'].where(df['delta'] < 0, 0)).fillna(0)

    avg_gain = wilder_smoothing(df['gain'], window)
    avg_loss = wilder_smoothing(df['loss'], window)

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    df['RSI_14'] = rsi
    df['RSI_14_ma'] = df['RSI_14'].rolling(window=14).mean()
    df['RSI_14_ma5'] = df['RSI_14'].rolling(window=5).mean()
    df['RSI_14_ma10'] = df['RSI_14'].rolling(window=10).mean()
    df['RSI_14_ma20'] = df['RSI_14'].rolling(window=20).mean()
    df['RSI_14_ma50'] = df['RSI_14'].rolling(window=50).mean()


    
    df['EMA_12'] = df['CLOSE'].ewm(span=12, min_periods=0, adjust=False, ignore_na=False).mean()
    df['EMA_26'] = df['CLOSE'].ewm(span=26, min_periods=0, adjust=False, ignore_na=False).mean()
    df['MACD'] = df['EMA_12'] - df['EMA_26']
    df['Signal_Line'] = df['MACD'].ewm(span=9, min_periods=0, adjust=False, ignore_na=False).mean()
    df['MACD_Histogram'] = df['MACD'] - df['Signal_Line']
    df['macd_trig'] = ((df['MACD_Histogram'] > 0)  & (df['MACD_Histogram'].shift(1) <= 0) & (df['MACD_Histogram'].shift(2) <0)).astype(int)
    df['macd_trig2'] = ((df['MACD_Histogram'] > 0)  & (df['MACD_Histogram'].shift(1) > 0) & (df['MACD_Histogram'].shift(2) <=0)& (df['MACD_Histogram'].shift(3) <0)& (df['MACD_Histogram'].shift(2) > df['MACD_Histogram'].shift(3) )).astype(int)
    df['macd_trig_DOWN'] = (((df['MACD_Histogram'] > 0))  & (df['MACD_Histogram'].shift(1) >df['MACD_Histogram']) & (df['MACD_Histogram'].shift(2) <df['MACD_Histogram'].shift(1)) & (df['MACD_Histogram'].shift(3) <df['MACD_Histogram'].shift(2)) & (df['MACD_Histogram'].shift(4) < df['MACD_Histogram'].shift(3)) & (df['MACD_Histogram'].shift(5) < df['MACD_Histogram'].shift(4))).astype(int)
    df['macd_trig_UP'] = (((df['MACD_Histogram'] < 0))  & (df['MACD_Histogram'].shift(1) < df['MACD_Histogram']) & (df['MACD_Histogram'].shift(2) > df['MACD_Histogram'].shift(1)) & (df['MACD_Histogram'].shift(3) >df['MACD_Histogram'].shift(2)) & (df['MACD_Histogram'].shift(4) > df['MACD_Histogram'].shift(3)) & (df['MACD_Histogram'].shift(5) > df['MACD_Histogram'].shift(4))).astype(int)
    
    
    
    df['BBBreak_upper'] = ((df['CLOSE']  > df['Upper_Band_BB'])  & (df['CLOSE'].shift(1) < df['Upper_Band_BB'].shift(1)) & (df['CLOSE'].shift(2) < df['Upper_Band_BB'].shift(2)) & (df['CLOSE'].shift(3) < df['Upper_Band_BB'].shift(3))).astype(int) 
    df['BBBreak_lower'] = ((df['CLOSE'] < df['Lower_Band_BB'])  & (df['CLOSE'].shift(1) > df['Lower_Band_BB'].shift(1)) & (df['CLOSE'].shift(2) > df['Lower_Band_BB'].shift(2)) & (df['CLOSE'].shift(3) > df['Lower_Band_BB'].shift(3))).astype(int)

    df['BBBreak_upper_tested'] = ((df['HIGH']  > df['Upper_Band_BB'])  & (df['HIGH'].shift(1) < df['Upper_Band_BB'].shift(1)) & (df['HIGH'].shift(2) < df['Upper_Band_BB'].shift(2)) & (df['HIGH'].shift(3) < df['Upper_Band_BB'].shift(3))).astype(int) 
    df['BBBreak_lower_tested'] = ((df['LOW'] < df['Lower_Band_BB'])  & (df['LOW'].shift(1) > df['Lower_Band_BB'].shift(1)) & (df['LOW'].shift(2) > df['Lower_Band_BB'].shift(2)) & (df['LOW'].shift(3) > df['Lower_Band_BB'].shift(3))).astype(int)
 
    
    df['BBBreak_upper_rsi14'] = ((df['CLOSE']  > df['Upper_Band_BB'])  & (df['CLOSE'].shift(1) < df['Upper_Band_BB'].shift(1)) & (df['CLOSE'].shift(2) < df['Upper_Band_BB'].shift(2)) & (df['CLOSE'].shift(3) < df['Upper_Band_BB'].shift(3)) & (df['RSI_14'] > 70)).astype(int) 
    df['BBBreak_lower_rsi14'] = ((df['CLOSE'] < df['Lower_Band_BB'])  & (df['CLOSE'].shift(1) > df['Lower_Band_BB'].shift(1)) & (df['CLOSE'].shift(2) > df['Lower_Band_BB'].shift(2)) & (df['CLOSE'].shift(3) > df['Lower_Band_BB'].shift(3))& (df['RSI_14'] < 30)).astype(int)


    
    df['rsi 7 ma break'] = ((df['RSI_7'] > df['RSI_7_ma'])  & (df['RSI_7'].shift(1) < df['RSI_7_ma'].shift(1)) & (df['RSI_7'].shift(2) < df['RSI_7_ma'].shift(2)) & (df['RSI_7'].shift(3) < df['RSI_7_ma'].shift(3))).astype(int)
    df['rsi 14 ma break'] = ((df['RSI_14'] > df['RSI_14_ma'])  & (df['RSI_14'].shift(1) < df['RSI_14_ma'].shift(1)) & (df['RSI_14'].shift(2) < df['RSI_14_ma'].shift(2)) & (df['RSI_14'].shift(3) < df['RSI_14_ma'].shift(3))).astype(int)    
    df['rsi 14 ma break5_20'] = ((df['RSI_14_ma5'] > df['RSI_14_ma20'])  & (df['RSI_14_ma5'].shift(1) < df['RSI_14_ma20'].shift(1)) & (df['RSI_14_ma5'].shift(2) < df['RSI_14_ma20'].shift(2)) & (df['RSI_14_ma5'].shift(3) < df['RSI_14_ma20'].shift(3))).astype(int)    

    
    
    
    df['rsi 14/7 ma break'] = ((df['RSI_7_ma'] > df['RSI_14_ma'])  & (df['RSI_7_ma'].shift(1) < df['RSI_14_ma'].shift(1)) & (df['RSI_7_ma'].shift(2) < df['RSI_14_ma'].shift(2)) & (df['RSI_7_ma'].shift(3) < df['RSI_14_ma'].shift(3))).astype(int)
    df['rsi 7 trig']=  ((df['BBBreak_lower']==1) & (df['RSI_7']<30)).astype(int)
    df['rsi 14 trig']=  ((df['BBBreak_lower']==1) & (df['RSI_14']<40)).astype(int)
    df['movingAVGcross1_buy'] = ((df['8DMA'] > df['21DMA'])  & (df['8DMA'].shift(1) < df['21DMA'].shift(1)) & (df['8DMA'].shift(2) < df['21DMA'].shift(2)) & (df['8DMA'].shift(3) < df['21DMA'].shift(3))).astype(int)
    df['movingAVGcross2_buy'] = ((df['20DMA'] > df['50DMA'])  & (df['20DMA'].shift(1) < df['50DMA'].shift(1)) & (df['20DMA'].shift(2) < df['50DMA'].shift(2)) & (df['20DMA'].shift(3) < df['50DMA'].shift(3))).astype(int)
    df['movingAVGcross1_sell'] = ((df['8DMA'] < df['21DMA'])  & (df['8DMA'].shift(1) > df['21DMA'].shift(1)) & (df['8DMA'].shift(2) > df['21DMA'].shift(2)) & (df['8DMA'].shift(3) > df['21DMA'].shift(3))).astype(int)
    df['movingAVGcross2_sell'] = ((df['20DMA'] < df['50DMA'])  & (df['20DMA'].shift(1) > df['50DMA'].shift(1)) & (df['20DMA'].shift(2) > df['50DMA'].shift(2)) & (df['20DMA'].shift(3) > df['50DMA'].shift(3))).astype(int)
    
    df['ABOVE_52week_high'] = ((df['CLOSE']  > df['52week_high']) & (df['CLOSE'].shift(1) < df['52week_high']) & (df['CLOSE'].shift(2) < df['52week_high']) &(df['CLOSE'].shift(3) < df['52week_high'])).astype(int)
    df['BELOW_52week_low'] = ((df['CLOSE']  < df['52week_low']) & (df['CLOSE'].shift(1) > df['52week_low']) & (df['CLOSE'].shift(2) > df['52week_low']) & (df['CLOSE'].shift(3) > df['52week_low'])).astype(int) 
    
    df['ABOVE_30day_high'] = ((df['CLOSE']  > df['30day_high']) & (df['CLOSE'].shift(1) < df['30day_high']) & (df['CLOSE'].shift(2) < df['30day_high']) &(df['CLOSE'].shift(3) < df['30day_high'])).astype(int)
    df['BELOW_30day_low'] = ((df['CLOSE']  < df['30day_low']) & (df['CLOSE'].shift(1) > df['30day_low']) & (df['CLOSE'].shift(2) > df['30day_low']) &(df['CLOSE'].shift(3) > df['30day_low'])).astype(int)
    
    df['ABOVE_7day_high'] = ((df['CLOSE']  > df['7day_high']) & (df['CLOSE'].shift(1) < df['7day_high']) & (df['CLOSE'].shift(2) < df['7day_high']) &(df['CLOSE'].shift(3) < df['7day_high'])).astype(int)
    df['BELOW_7day_low'] = ((df['CLOSE']  < df['7day_low']) & (df['CLOSE'].shift(1) > df['7day_low']) & (df['CLOSE'].shift(2) > df['7day_low']) &(df['CLOSE'].shift(3) > df['7day_low'])).astype(int)

    df['ABOVE_10day_high'] = ((df['CLOSE']  > df['10day_high']) & (df['CLOSE'].shift(1) < df['10day_high']) & (df['CLOSE'].shift(2) < df['10day_high']) &(df['CLOSE'].shift(3) < df['10day_high'])).astype(int)
    df['BELOW_10day_low'] = ((df['CLOSE']  < df['10day_low']) & (df['CLOSE'].shift(1) > df['10day_low']) & (df['CLOSE'].shift(2) > df['10day_low']) &(df['CLOSE'].shift(3) > df['10day_low'])).astype(int)
     
    df['sure_BUY']= ((df['CLOSE']  > df['OPEN']) & (df['CLOSE']< df['Lower_Band_BB'])).astype(int)
    df['sure_SELL']= ((df['CLOSE']  < df['OPEN']) & (df['CLOSE']> df['Upper_Band_BB'])).astype(int)

    
    dfs1_filtered[symbol] = df 

    
    
import pandas as pd

# Assuming 'dfs1' is the dictionary of data frames
# Initialize an empty list to store the data frames
dfs_list = []
combined_df=pd.DataFrame()
# Iterate over each data frame in the dictionary and append it to the list
for df in dfs1_filtered.values():
    dfs_list.append(df)

# Concatenate all data frames in the list
combined_df = pd.concat(dfs_list)

# Reset the index to avoid duplicate index values
combined_df = combined_df.reset_index(drop=False)


#selected_column=['TIMESTAMP','SYMBOL_y','RSI_7','RSI_14','CLOSE','BBBreak_upper','BBBreak_lower','BBBreak_upper_50','BBBreak_lower_50','BBBreak_upper_7','BBBreak_lower_7','TEST_50','TEST_7','max_15DAY','max_30DAY','15 greater than abs 4%','30 greater than abs 4%','15 greater than 4%','5 greater than 1%']  # Replace with your desired column names
selected_column=['TIMESTAMP',
                 'SYMBOL',
                 'RSI_7',
                 'RSI_14',
                 'CLOSE',
                 'Latest_Close_Price',
                 'Percentage_Difference',
                 'Bullish_Engulfing',
                 'BBBreak_upper_rsi14',
                 'BBBreak_lower_rsi14',
                 'sure_BUY',
                 'sure_SELL',
                 'Bearish_Engulfing',
                 'AO_BUY',
                 'AO_SELL',
                 'macd_trig_DOWN',
                 'macd_trig_UP',
                 'BBBreak_upper',
                 'BBBreak_lower',
                 'BBBreak_upper_tested',
                 'BBBreak_lower_tested',
                 'rsi 7 ma break',
                 'rsi 14 ma break',
                 'rsi 14/7 ma break',
                 'rsi 7 trig',
                 'rsi 14 trig',
                 'movingAVGcross1_buy',
                 'movingAVGcross2_buy',
                 'movingAVGcross1_sell',
                 'movingAVGcross2_sell',
                 'ABOVE_52week_high',
                 'BELOW_52week_low',
                 'ABOVE_30day_high',
                 'BELOW_30day_low',
                 'ABOVE_7day_high',
                 'BELOW_7day_low',
                 'ABOVE_10day_high',
                 'BELOW_10day_low',
                 
                 
                 ]  # Replace with your desired column names
selected_df = combined_df[selected_column]
selected_df.to_csv('D:/4.Daily_stocks/Full_stocks.csv')
import os
import pandas as pd

# Ensure the directory exists, if not create it
save_path = 'D:/4.Daily_stocks'
if not os.path.exists(save_path):
    os.makedirs(save_path)

# Sample dataframe (you'll have your dataframe here)
# df = pd.read_csv('your_file.csv')

# List of columns to filter
columns_to_filter = ['BBBreak_upper_rsi14',
                     'BBBreak_lower_rsi14',
                     'sure_BUY',
                     'sure_SELL',
                     'Bullish_Engulfing',
                     'Bearish_Engulfing',
                     'AO_BUY',
                     'AO_SELL',
                     'macd_trig_DOWN', 
                     'macd_trig_UP',
                     'BBBreak_upper', 
                     'BBBreak_lower', 
                     'BBBreak_upper_tested',
                     'BBBreak_lower_tested', 
                     'movingAVGcross1_buy',
                     'movingAVGcross2_buy', 
                     'movingAVGcross1_sell', 
                     'movingAVGcross2_sell',
                     'ABOVE_52week_high', 
                     'BELOW_52week_low', 
                     'ABOVE_30day_high',
                     'BELOW_30day_low',
                     'ABOVE_7day_high', 'BELOW_7day_low', 
                     'ABOVE_10day_high', 'BELOW_10day_low']

# Loop through each specified column
for column in columns_to_filter:
    # Filter rows where the column value is 1
    filtered_df = selected_df[selected_df[column] == 1]
    
    # If the filtered dataframe is not empty and has a 'TIMESTAMP' and 'CLOSE' column, extract required columns
    if not filtered_df.empty and 'TIMESTAMP' in filtered_df.columns and 'CLOSE' in filtered_df.columns:
        # Extracting the latest close value
        latest_close = filtered_df['CLOSE'].iloc[-1]
       # Select required columns
        filtered_df_final = filtered_df[['TIMESTAMP', 'SYMBOL','CLOSE','Latest_Close_Price','Percentage_Difference']]
        
        # Sort by 'TIMESTAMP' in descending order (newest to oldest)
        filtered_df_final_sorted = filtered_df_final.sort_values(by='TIMESTAMP', ascending=False)
        
        # Define the full file path
        file_path = os.path.join(save_path, f"{column.replace(' ', '_')}_filtered.csv")
        
        # Save the sorted dataframe with required columns to the specified directory
        filtered_df_final_sorted.to_csv(file_path, index=False)

import os
import pandas as pd

# Ensure the directory exists, if not create it
save_path = 'D:/4.Daily_stocks/'
if not os.path.exists(save_path):
    os.makedirs(save_path)

# Sample dataframe (you'll have your dataframe here)
# selected_df = pd.read_csv('your_file.csv')

# List of columns to filter
columns_to_filter = ['sure_BUY','sure_SELL','Bullish_Engulfing','Bearish_Engulfing','AO_BUY','AO_SELL','macd_trig_DOWN', 'macd_trig_UP', 'BBBreak_upper', 'BBBreak_lower', 
                     'BBBreak_upper_tested', 'BBBreak_lower_tested', 'rsi 7 ma break', 
                     'rsi 14 ma break', 'rsi 14/7 ma break', 'rsi 7 trig', 'rsi 14 trig', 
                     'movingAVGcross1_buy', 'movingAVGcross2_buy', 'movingAVGcross1_sell', 
                     'movingAVGcross2_sell', 'ABOVE_52week_high', 'BELOW_52week_low', 
                     'ABOVE_30day_high', 'BELOW_30day_low', 'ABOVE_7day_high', 'BELOW_7day_low', 
                     'ABOVE_10day_high', 'BELOW_10day_low']

# Create Excel Writer object
excel_path = os.path.join(save_path, "filtered_data.xlsx")

# Dictionary to hold dataframes
dfs_to_save = {}

# Loop through each specified column
for column in columns_to_filter:
    # Filter rows where the column value is 1
    filtered_df = selected_df[selected_df[column] == 1]
    
    # If the filtered dataframe is not empty and has required columns
    if not filtered_df.empty and 'TIMESTAMP' in filtered_df.columns and 'CLOSE' in filtered_df.columns:
        # Extract the latest close value
        latest_close = filtered_df['CLOSE'].iloc[-1]
        
        # Select required columns
        filtered_df_final = filtered_df[['TIMESTAMP', 'SYMBOL', 'CLOSE', 'Latest_Close_Price', 'Percentage_Difference']]
        
        # Sort by 'TIMESTAMP' in descending order (newest to oldest)
        filtered_df_final_sorted = filtered_df_final.sort_values(by='TIMESTAMP', ascending=False)
        
        # Replace invalid characters in the sheet name
        sheet_name = column.replace(' ', '_').replace('/', '_')  # Replacing '/' with '_'
        
        # Add the sorted dataframe to the dictionary with modified sheet name as the key
        dfs_to_save[sheet_name] = filtered_df_final_sorted

# Save dataframes to Excel file with each dataframe as a separate sheet
with pd.ExcelWriter(excel_path) as writer:
    for sheet_name, df_to_save in dfs_to_save.items():
        # Convert 'TIMESTAMP' to datetime format
        df_to_save['TIMESTAMP'] = pd.to_datetime(df_to_save['TIMESTAMP'])
        
        # Format 'TIMESTAMP' to only include year, month, and day
        df_to_save['TIMESTAMP'] = df_to_save['TIMESTAMP'].dt.strftime('%d-%m-%Y')
        
        # Save the dataframe to Excel with the formatted TIMESTAMP column
        df_to_save.to_excel(writer, sheet_name=sheet_name, index=False)

